In [100]:
import nltk            # natural language tool kit
import numpy as np     # support for large data structures
import pandas as pd    # data structure support
import string          # various string functions
import difflib         # classes and functions for comparing sequences
from sklearn.metrics import log_loss    # used in measurement / scoring
from sklearn.metrics import classification_report   # among other things, provides accuracy and f1
from numpy import dot
from numpy.linalg import norm

from importlib import reload

# Libraries for Doc2Vec processing
from gensim import models, corpora, similarities
from gensim.models.doc2vec import TaggedDocument, Doc2Vec
from nltk.tokenize import TweetTokenizer
import pyemd
import csv
import multiprocessing

cores = multiprocessing.cpu_count()
print(cores)

import pos3
from importlib import reload 

# Load the Penn Treebank Corpus which will serve as our training set.
#corpus = nltk.corpus.treebank
corpus = nltk.corpus.ptb
print('There are %d sentences in the corpus.' % len(corpus.tagged_sents()))
print('The first sentence is:')
corpus.tagged_sents()[0]

8
There are 73451 sentences in the corpus.
The first sentence is:


[('In', 'IN'),
 ('American', 'JJ'),
 ('romance', 'NN'),
 (',', ','),
 ('almost', 'RB'),
 ('nothing', 'NN'),
 ('rates', 'VBZ'),
 ('*-2', '-NONE-'),
 ('higher', 'JJR'),
 ('than', 'IN'),
 ('what', 'WP'),
 ('the', 'DT'),
 ('movie', 'NN'),
 ('men', 'NNS'),
 ('have', 'VB'),
 ('called', 'VBN'),
 ('*T*-1', '-NONE-'),
 ('``', '``'),
 ('*', '-NONE-'),
 ('meeting', 'NN'),
 ('cute', 'JJ'),
 ("''", "''"),
 ('--', ':'),
 ('that', 'DT'),
 ('is', 'VBZ'),
 (',', ','),
 ('boy-meets-girl', 'NN'),
 ('seems', 'VBZ'),
 ('more', 'RB'),
 ('adorable', 'JJ'),
 ('if', 'IN'),
 ('it', 'PRP'),
 ('does', 'VBZ'),
 ("n't", 'RB'),
 ('take', 'VB'),
 ('place', 'NN'),
 ('in', 'IN'),
 ('an', 'DT'),
 ('atmosphere', 'NN'),
 ('of', 'IN'),
 ('correct', 'JJ'),
 ('and', 'CC'),
 ('acute', 'JJ'),
 ('boredom', 'NN'),
 ('.', '.')]

In [142]:
train = pd.read_csv('Data/train_split.csv')
del train["Unnamed: 0"]
train.head(20)

,id,qid1,qid2,question1,question2,is_duplicate
0,57674,5725,38477,How can I get a list of my Gmail accounts?,How do I find my list of GMail addresses?,1
1,185568,283254,283255,What are the symptoms of child abuse?,What are signs of child abuse?,1
2,359111,400936,488715,What are some examples of selfishness?,Do we all occasionally tend to emotional masoc...,0
3,204216,281934,307050,Is Qnet a scam?,Where is the registered office of Qnet in Mumbai?,0
4,5649,11103,11104,What do Americans think about Donald Trump?,What do you think about Donald Trump pick?,1
5,173796,268021,268022,How can we cure molluscum?,How can I cure mysthenu?,0
6,341162,160540,468955,What is the difference between a Java applet a...,What is the difference of Java applet and Java...,0
7,288327,409266,409267,Is meditation dangerous and should I do it?,Is meditation very dangerous?,1
8,302830,425865,425866,What are good gift ideas for a dad in his late...,What are some good gift ideas for my girlfrien...,0
9,14588,27925,27926,Can a restaurant be sued for serving meat to v...,How is vegetarian feta cheese served?,0


In [41]:
qpair = train.iloc[[0]]
print(type(qpair["question1"].item()))

<class 'str'>


In [43]:
#with open('Data/train_split.csv', 'rU') as f:
#    f.next()
#    reader = csv.reader(f)
#    qpairs = list(reader)
   
tknzr = TweetTokenizer()
qcorp = []
qdict = {}
for i in range(len(train)):
    qpair = train.iloc[[i]]
    qid = qpair["qid1"].item()
    qwords = tknzr.tokenize(qpair["question1"].item())
    q = TaggedDocument(words=qwords, tags=["QID_"+str(qid)])
    qcorp.append(q)
    qdict["QID_"+str(qid)] = qwords
    qid = qpair["qid2"].item()
    if type(qpair["question2"].item()) is str:
        qwords = tknzr.tokenize(qpair["question2"].item())
        q = TaggedDocument(words=qwords, tags=["QID_"+str(qid)])
        qcorp.append(q)
        qdict["QID_"+str(qid)] = qwords
    else:
        print(qpair["question1"])
        print(qpair["question2"])


26691    How can I develop android app?
Name: question1, dtype: object
26691    NaN
Name: question2, dtype: object


In [78]:
model = Doc2Vec(size=100, window=3, negative=5, sample=1e-4, min_count=2, workers=cores, alpha=0.025, min_alpha=0.020, iter=20)
model.build_vocab(qcorp)
model.train(qcorp, total_examples=model.corpus_count, epochs=model.iter)

98911406

In [79]:
model.save("doc2vec.model")

In [51]:
model = Doc2Vec.load("doc2vec.model")

In [80]:
vectors = model.docvecs

In [53]:
dv_results = []
for i in range(len(qpairs)):
    qid1 = "QID_"+str(qpairs[i][1])
    qid2 = "QID_"+str(qpairs[i][2])
    dv_similarity = vectors.similarity(qid1, qid2)
    dv_results.append([i, dv_similarity])
    #qpairs[i].extend([score])

NameError: name 'qpairs' is not defined

In [27]:
actuals = np.array(train['is_duplicate'])
n_dv_results = np.array(dv_results)

predictions_dv = n_dv_results[:,1]

In [28]:
# build a function to run scoring
# function takes an array of actuals, predicted values (as percentages, not absolute), and a probability 
# probability represents value over which over which we assume = 1)
# percentages are converted to absolute values (0 or 1) with the function

def score(actuals, predictions, probability):
    score_sm = log_loss(actuals, predictions)
    print ('log loss score is: %3f' %(score_sm))
    
    n_abs = np.where(predictions > probability, 1, 0)
    
    total_wrong = np.sum(np.not_equal(actuals, n_abs))
    print ('number of incorrect predictions is: %3d' %(total_wrong))
    total = len(actuals)
    correct = total - total_wrong
    print ('total: %3d  correct: %3d  accuracy: %3.2f \n' %(total, correct, 1.0*correct/total))
    print classification_report(actuals, n_abs)

In [29]:
score(actuals, predictions_dv, .50)

log loss score is: 0.661802
number of incorrect predictions is: 136267
total: 404351  correct: 268084  accuracy: 0.66 

             precision    recall  f1-score   support

          0       0.83      0.59      0.69    255045
          1       0.53      0.79      0.63    149306

avg / total       0.72      0.66      0.67    404351



In [25]:
for i in range(25):
    print qpairs[i]

['0', '1', '2', 'What is the step by step guide to invest in share market in india?', 'What is the step by step guide to invest in share market?', '0']
['1', '3', '4', 'What is the story of Kohinoor (Koh-i-Noor) Diamond?', 'What would happen if the Indian government stole the Kohinoor (Koh-i-Noor) diamond back?', '0']
['2', '5', '6', 'How can I increase the speed of my internet connection while using a VPN?', 'How can Internet speed be increased by hacking through DNS?', '0']
['3', '7', '8', 'Why am I mentally very lonely? How can I solve it?', 'Find the remainder when [math]23^{24}[/math] is divided by 24,23?', '0']
['4', '9', '10', 'Which one dissolve in water quikly sugar, salt, methane and carbon di oxide?', 'Which fish would survive in salt water?', '0']
['5', '11', '12', 'Astrology: I am a Capricorn Sun Cap moon and cap rising...what does that say about me?', "I'm a triple Capricorn (Sun, Moon and ascendant in Capricorn) What does this say about me?", '1']
['6', '13', '14', 'Shou

In [81]:
wordvectors = model.wv

In [140]:
print(wordvectors.most_similar(["boosters"]))

[('diffusers', 0.6048799753189087), ('cancerides', 0.5876442193984985), ('aspirated', 0.5872788429260254), ('cinchers', 0.5846953392028809), ('arresters', 0.5789477825164795), ('pumps', 0.5783033967018127), ('meditations', 0.5774851441383362), ('hexagonal', 0.5761784911155701), ('fabricators', 0.5706850290298462), ('12mm', 0.5669615864753723)]


In [83]:
test = "QID_400936"
print(qdict[test])
for q in vectors.most_similar(["QID_400936"]):
    print(q, qdict[q[0]])

['What', 'are', 'some', 'examples', 'of', 'selfishness', '?']
('QID_378642', 0.941085159778595) ['What', 'is', 'a', 'Oligarchy', '?', 'What', 'are', 'some', 'examples', '?']
('QID_346301', 0.940555214881897) ['What', 'are', 'some', 'mannerism', 'examples', '?']
('QID_337169', 0.9387899041175842) ['What', 'are', 'some', 'examples', 'of', 'plasticizer', '?']
('QID_65404', 0.935355544090271) ['What', 'are', 'polysaccharides', '?', 'What', 'are', 'some', 'examples', '?']
('QID_427674', 0.9343449473381042) ['What', 'are', 'some', 'examples', 'of', 'mutagens', '?']
('QID_85600', 0.9323787689208984) ['What', 'are', 'examples', 'of', 'disaccharides', '?']
('QID_258618', 0.9319553971290588) ['What', 'are', 'some', 'examples', 'of', 'dehumanization', '?']
('QID_319471', 0.9300248622894287) ['What', 'is', 'sequent', 'occupance', '?', 'What', 'are', 'some', 'examples', '?']
('QID_113789', 0.9275696277618408) ['What', 'are', 'examples', 'of', 'Nematoda', '?']
('QID_284403', 0.9272573590278625) ['Wh